# Analysis of the SQL database of a subscription book reading service

## Goal: formulate a value proposition for a new product

Tasks: 
1. Count how many books were published after January 1, 2000;
2. For each book, count the number of reviews and the average rating;
3. Identify the publishing house that published the largest number of books thicker than 50 pages - this way you will exclude brochures from the analysis;
4. Determine the author with the highest average book rating - only consider books with 50 or more ratings;
5. Calculate the average number of reviews from users who gave more than 48 ratings.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import text, create_engine
# set parameters
db_config = {'user': '.....', # username
'pwd': '.....', # pasword
'host': '....',
'port': ..., # connection port
'db': '......'} # название базы данных
connection_string = '...'.format(**db_config)
# save the connector
engine = create_engine(connection_string, connect_args={'....'})
# to execute an SQL query, we use Pandas
#select first five line from all table
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


The data is well structured. The existing description of the database corresponds to reality

In [6]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
# Task 1
# Number of books published after January 1, 2000
query = '''SELECT COUNT(*) 
FROM books AS b
WHERE publication_date>'2000-01-01'
                                       '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Not much. I hope they are worth it for users to pay for them.

In [8]:
#Task 2
# For each book, determine the number of reviews and the average rating
query = '''SELECT b.book_id AS book,
                  b.title AS title,
                 COUNT(DISTINCT rew.review_id) as count_reviews,
                 AVG(r.rating) AS mean_rating
           FROM books AS b LEFT JOIN ratings AS r ON b.book_id = r.book_id 
                LEFT JOIN reviews AS rew ON b.book_id = rew.book_id
           GROUP BY b.book_id;
                                       '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)




,book,title,count_reviews,mean_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


Looks like the books are interesting to someone.

In [9]:
#Nask 3
#publishing house that has published the largest number of books thicker than 50 pages
query = '''SELECT  p.publisher,
                   COUNT(b.book_id) as number_books
           FROM books AS b 
                LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id 
           WHERE b.num_pages>50
           GROUP BY p.publisher_id
           ORDER BY number_books DESC
           LIMIT 1;
                                       '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,number_books
0,Penguin Books,42


Penguin Books is the leader. The number of books is not huge. But I hope they are interesting.

In [10]:
#Task 4
query = '''SELECT a.author_id,
                  a.author,
                  AVG(r.avg_ratings) as auth_avg_ratings
           FROM authors AS a INNER JOIN books AS b ON a.author_id = b.author_id
                             INNER JOIN  (
                                          SELECT r.book_id AS book_id, AVG(r.rating) as avg_ratings
                                          FROM ratings AS r 
                                          GROUP BY r.book_id
                                          HAVING COUNT(r.rating_id)>50
                                          ) AS r ON r.book_id=b.book_id
           GROUP BY a.author_id
           ORDER BY auth_avg_ratings DESC
           LIMIT 1;
                                       '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,author_id,author,auth_avg_ratings
0,236,J.K. Rowling/Mary GrandPré,4.283844


In [11]:
#Task 5
#average number of reviews from users who gave more than 48 ratings
query = '''

           SELECT AVG(count_rev) AS avg_count_rev
           FROM(
           SELECT COUNT(rev.review_id) AS count_rev
           FROM reviews AS rev
           WHERE rev.username in (                
                                      SELECT r.username AS username 
                                      FROM ratings AS r 
                                      GROUP BY r.username
                                      HAVING COUNT(rating_id)>48
                                                       
           
                                     )
           
          GROUP BY rev.username) AS count_rev;
                                       '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_count_rev
0,24.0


The users seem to be quite active.

## Concluions.

The data in the tables is well structured and reflects a lot of important information about the books. First of all, you need to appreciate users for their dedicated work in writing reviews. With a modest number of books, Users read, rate and write reviews. The resulting assessment can be relied upon for further development. Moreover, there are a lot of books in the world, we can probably work on increasing the number of books in the service. To do this, it is worth exploring the assortment of the Penguin Books publishing house. The most favorite author is J.K. Rowling. Readers love the classics.